In [1]:
from SALib.sample import saltelli
from SALib.analyze import sobol
from SALib.test_functions import Ishigami
import numpy as np

from math import sin, cos, pi

# 1. Анализ чувствительности по Соболю

`Зафиксируйте какую-либо многомерную скалярную функцию. Реализуйте для неё анализ чувствительности по методу Соболя
на Python с использованием библиотеки SALib. Проведите анализ чувствительности, проверьте сходимость, измерьте тайминги.`

В качестве анализируемой функции возьмем функцию, выражающую гравитационный момент по оси **X**, действующий на твердое тело, в частности, космический аппарат (далее КА). В общем случае, гравитационный момент в векторном виде выражается следующей формулой:

$$
\vec{M} = 3 \frac{\mu}{R^3} \cdot (\vec{\eta} \times \textbf{I} \vec{\eta}),
$$

где $\mu \approx 398600 \frac{км^3}{c^2}$ - константа, гравитационный параметр Земли;

${R}$ - расстояние от центра Земли до центра масс КА;

$\vec{\eta}$ - единичный вектор направления от центра Земли к центру масс КА, записанный в связанной с КА системе координат;

$\textbf{I} = \begin{pmatrix}
I_{X} & -I_{XY} & -I_{XZ}\\
-I_{XY} & I_{Y} & -I_{YZ}\\
I_{XZ} & -I_{YZ} & I_{Z}\\
\end{pmatrix}$ - тензор инерции космического аппарата.

В проекции на ось **X**, связанную с КА, гравитацонный момент можно записать в следующем виде:

$$
M_X = 3 \frac{\mu}{R^3} cos\vartheta [\frac{1}{2} (I_Y - I_Z) sin2\gamma cos\vartheta - I_{YZ}cos2\gamma cos\vartheta - (I_{XZ}cos\gamma + I_{XY}sin\gamma)sin\vartheta],
$$

где $\gamma, \vartheta$ -  углы крена и тангажа КА соответственно.

Заметим, что здесь присутствует всего 2 угла. От третьего угла, рысканья ($\psi$ - вращение вокруг вертикальной оси **Y**), гравитационный момент не зависит.

Для расчетов возьмем радиус Земли, равным $6370км$. В качестве КА рассмотрим **МКС**: высота орбиты ~$400км$, моменты инерции взяты отсюда:

https://athena.ecs.csus.edu/~grandajj/ME296M/RevAB_Volume%20II%20Signed_updated.pdf

> Код программы можно посмотреть в файле `sobol.py`

### Результаты и выводы:

> * **Время исполнения**. Было взято 65536 точек. Результаты по времени исполнения (в секундах) приведены в таблице ниже:
> |Sample generation|Model evaluation|Sobol analysis|
> |---|---|---|
> |1.9|1.03|5.75|
>
> Модель взял простенькую. Ничего не придумал сложнее.  Поэтому время для ее расчетов самое минимальное оказалось. Это без всяких оптимизаций.

> * **Результаты**. Результаты анализа чувствительности следующие:
> 
> **S1:** $\ \ \ [0.6469\ \ \ 0.\ \ \ 0.]$
> 
> **ST:** $\ \ \ [0.9999\ \ \ 0.\ \ \ 0.3531]$
> 
> **x1-x2:** $\ \ \ .0$
> 
> **x1-x3:** $\ \ \ 0.3531$
> 
> **x2-x3:** $\ \ \ -0.0$

> * **Сходимость**. Полностью всех положительных значений не удалось достичь. При 65536 точках остался отрицательный знак у коэффициента чувствительности второго порядка **x2-x3**. При этом это очень маленькое число. Возможно, знак связан с точностью расчетов.
> Если же увеличивал чисто сэмплируемых точек, то появлялся отрицательный знак у третьего коэффициента чувствительности первого порядка (угол тангажа) и у коэффициента чувствительности второго порядка **x1-x2**.

> * **Анализ рельутатов.**
> Тут не смог осознать до конца результаты. 
> 
> **Коэффициенты первого порядка**
>
> Ненулевой коэффициент чувствительности только первый - для угла крена. 
> 
> Второй коэффициент нулевой, т.к. соответствующий угол не присутствует в финальной формуле, что, собственно, логично. Не отбрасывал его по 2 причнам: в исходной формуле, записанной в векторном виде, этот угол есть + ориентация твердого тела описывается тремя, а не двумя углами.
> 
> Третий коэффициент очень маленький. Это значит мы можем его отбросить или зафиксировать. Но отбросить его не можем, т.к. он в качестве сомножителя везде. Значит только фиксируем.
> 
> **Коэффициенты полного порядка** 
>
> Только один коэффициент нулевой - второй. А два других ненулевые. По ним получается, что третий угол тангажа мы не можем отбросить или зафиксировать, что противоречит выводам по коэффициентам первого порядка.
>
> **Коэффициенты второго порядка** 
>
> Тут все логично. Ненулевой коэффициент только у **x1-x3**. Два остальных нулевые (с участием **x2**), т.к. вторая переменная не участвует в функции.
>

# 2.  Ускорение Python. Параллелизм.

* `Ускорьте вычисления Python с использованием любой из имеющихся возможностей (PyBind11, ctypes, cython, numba)`
* `Попробуйте добавить параллелизм в вычисления`

В качестве программы, которую будем ускорять, взял программу расчета числа **e** (сумма бесконечного ряда). 

При этом, при использовании языка **си** или **numba** - факториал большого числа не хочет вычисляться. Поэтому, чтобы увеличить время, просто вычисляю много раз число **e**, все результаты суммирую и ответ усредняю.

> См. файлы `calc_e.py` и `calc_e.c`

Результаты времени выполнения расчетов для разных случаев приведены ниже в таблице:

|Метод|Время исполнения программы, с|
|---|---|
|Без оптимизаций|**39.41**|
|**numba (njit)** 1-ый запуск|**0.71**|
|**numba (njit)** 2-ой запуск|**0.44**|
|**ctypes**|**0.48**|
|**ctypes** с **omp**|**0.10**|

### Выводы

* Самым быстрым оказалось использование **ctypes** с распараллеливаванием программы внутри. Скорость выполнения почти в 400 раз больше, чем без оптимизаций.
* **njit** и **ctypes** без параллельных кусков показали примерно одинаковые результаты. Время выполнения почти в 100 раз меньше, чем без оптимизаций.